## Dieser Teil des Notebooks befasst sich mit den Aufgabenteilen 1A und 1B 

Im ersten Schritt sollen hierbei die Daten von einer Wikipedia-Schnittstelle heruntergeladen und extrahiert werden. 
Die Dump-Dateien, welche alle nötigen Wikipedia-Inhalte umfassen, können über das Ausführen der in der **Readme**-Datei aufgelisteten Befehlen gewählt und geladen werden.
Zur Auswahl stehen hier 1) eine Dumpdatei mit einer kleineren Größe von ca. 5GB (im Prozess entstehend) für ein effizientes Arbeiten und 2) die allumfassende Dumpdatei mit einer Größe von ca. 28GB (im Prozess entstehend)

Sobald über die README-Datei und die Bash-Skripte u.a. das Aufsetzen der Datenstruktur erfolgt ist, kann eine erste Extraktion des gewählten Dumps stattfinden:

### Extraktion des Dumps:

Die Bibliothek "MediaWiki XML Processing" oder auch "mwxml" ermöglicht ein vereinfachtes Streamen XML-Dumps über eine Abstrahierung.
Über den mwxml.Dump werden die Wikipediaartikel auf eine iterative Weise über mwxml.Page‘s zugänglich gemacht.
Im Folgenden werden über die angegebene Datei (entweder die kleine oder große XML-Datei) ein mwxml-Dump erstellt, welcher eben diese Attribute bereitstellt.
Um die Funktionalität und Fehlerfreiheit zu prüfen werden im Folgenden zudem die Info der Seite und der Name der untersuchten Datenbank ausgegeben. Zu erwarten wären hier Hinweise auf die deutsche Wikipediafassung.

In [1]:
# Importieren der Bibliotheken und Module
import mwxml # Für Untersuchung der XMLs
import re # Für spätere regex-Operationen
from tqdm import tqdm # Für die Integration von Fortschrittsbannern, Ladebalken etc.

dump = mwxml.Dump.from_file(open("../data/dewiki-20220520-pages-articles-multistream1.xml")) #dewiki-20220620-pages-articles-multistream.xml als Alternative, falls großer Dump genutzt werden soll.
print(dump.site_info.name, dump.site_info.dbname) # Ausgabe der Informationen zum Dump

Wikipedia dewiki


Im Folgenden wird vorläufig eine später verwendete Methodik und Funktion zur weiteren Verbesserung der Firmenklassifikation implementiert.
Für eine spätetere Ausräumung von Unsicherheiten in deutscher Wiki werden sowohl deutsches NER-Modell auf deutscher Wiki und englisches Modell auf englischer Wiki genutzt. 
Auch mit deutschem Modell auf rein deutscher Wikipedia möglich.

In [2]:
!python -m spacy download en_core_web_sm # Herunterladen des englischen Spacy-Sprachmodells per Bash-Skript - erfahrungsgemäß bessere Zuverlässigkeit.
!python -m spacy download de_core_news_sm # Herunterladen des deutschsprachigen Spacy-Sprachmodells per Bash-Skript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.1 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 4.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


Nachdem die nötigen Modelle heruntergeladen wurden, kann nun die Funktion zur Verbesserung der Genauigkeit implementiert werden.

In [3]:
# Firmenerkennung über NER
import spacy # spacy ist eine Bibliothek für fortgeschrittene Natürliche Sprachverarbeitung
import wikipediaapi # Vorteil: sowohl für 'de' als auch 'en' nutzbar: die wikipediapi ist eine Implementierung der Wikipedia-Api als eine "Wikipedia Python API". So können sehr effizient Zusammenfassungen und Seiten zu gesuchten Titeln angefordert werden.


en_nlp = spacy.load("en_core_web_sm") # Laden des englischen Spacy-Modells für NER
de_nlp = spacy.load("de_cor_news_sm") # Laden des deutschen Spacy-Modells für NER
en_wiki = wikipediaapi.Wikipedia('en') # Laden der englischen wikipediaapi-Version
de_wiki = wikipediaapi.Wikipedia('de') # Laden des deutschen wikipediaapi-Version


def reduceUncertainty(page_title): # Diese Funktion soll abgerufen werden können, falls bei einer erkannten Entität nicht klar ist, ob es sich um eine Firma handelt. Hierdurch sollen Unsicherheiten reduziert werden.
    en_page = en_wiki.page(page_title) # Das Funktionsattribut page_title enthält den gesuchten Firmenname/Wikipediatitel und wird hier in der englischen API gesucht. 
    de_page = de_wiki.page(page_title) # Das Funktionsattribut page_title enthält den gesuchten Firmenname/Wikipediatitel und wird hier über die deutsche API gesucht. 
    org_list = [] # Diese Liste wird leer erklärt, um später alle erwähnten Organisationen in dem Betrefflichen Firmenartikel zu erkennen.

    if not en_page.exists(): # Englische Wikipedia-Seite existiert nicht
        if not de_page.exists(): # Deutsche  Wikipedia-Seite existiert auch nicht
            print("01 - ", page_title) 
            return True # gibt True("company exists") zurück, wenn die Seite nicht gefunden werden kann. So werden nur Seiten entfernt über die wir Zusatzwissen erlangen konnten und am wenigsten drastisch eingegriffen. Je nach Business Case kann dies anders Implementiert werden.
        else: # Deutsche Seite existiert aber Englische Seite existiert nicht - in diesem Fall muss idealerweise ein deutsches NER-Modell verwendet werden
            doc = de_nlp(de_page.summary) # Die Zusammenfassung sollte in den meisten Fällen für die Urteilsfällung ausreichen
            for ent in doc.ents: # Alle erkannten Entitäten werden hier gesammelt
                if (ent.label_ == 'ORG') and (page_title in ent.text): # ... Und anschließend gefiltert: Zunächst sollen nur erkannte Organisationen behalten werden, welche zudem in ihrem Eigennamen den Titel der untersuchten Seite als String enthalten. Zukünftig könnten hier erweiterte Regex-Operationen verwendet werden.
                    org_list.append(ent) # Nur diese, den Kriterien unterliegende Entitäten, werden der Liste von erkannten Organisationen hinzugefügt.
            if(len(org_list)>0): 
                print("11 - ", page_title)
                return True
            else: 
                print("10 - ", page_title)
                return False

    else: # Englische Seite existiert und wird aufgrund Informationsgehalt sowie zuverlässigkeit des englischen Sprachmodells bevorzugt.
        doc = en_nlp(en_page.summary) # hier findet das selbige Vorgehen wie oben statt, nur mit einem englischen NER-Modell, da dieses idealerweise für die englischen Wikipediaseiten verwendet werden sollte.
        for ent in doc.ents: 
          if (ent.label_ == 'ORG') and (page_title in ent.text):
            org_list.append(ent)
        if(len(org_list)>0):
            print("11 - ", page_title)
            return True
        else: 
            print("10 - ", page_title)
            return False

/Users/amonschneider/opt/anaconda3/envs/forschungsseminarenv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Das Obiges Skript durchläuft also den folgenden logischen Prozess bzw. Algorithums:

**existiert Artikel im Englischen Wiki?**
- **JA:** versuche Organisationen mit englischem Modell zu erkennen
- **NEIN:** existiert Artikel im Deutschen Wiki? 
    - **JA:** versuche Organisationen mit deutschem Modell zu erkennen
    - **NEIN:** aus Toleranz als Firma oder nicht Firma zählen.

Im Folgenden wird diese Methode kurz getestet, um Sie später in der Sammlung von Firmennamen anwenden zu können:

In [2]:
# Tests:

page = en_wiki.page("Aldi") # Benennung der gesuchten Wikipediaseite als "Aldi"

print(page.exists()) # Überprüfung ob Seite existiert

print("Page - Summary: %s" % page.summary[0:60]) # Ausgabe eines Zusammenfassungteils

doc = en_nlp(page.summary) # Laden der Seitenzusammenfassung in englisches NLP-Modell

for ent in doc.ents: # Iterieren über erkannte Entitäten
    print(ent.text, ent.label_) # Ausgabe der Entitätenart und des Titels

# Nun kann sich ein Bild gemacht werden, ob die Zusammenfassung Entitäten liefert, die darauf hindeuten, dass es sich bei "Aldi" um eine Firma handelt.

NameError: name 'en_wiki' is not defined

### Sammeln der Firmennamen aus Dump:

Nun können mithilfe der obigen Funktionen und mithilfe von im Folgenden erklärten regex-Mustern mit erhöhter Zuverlässigkeit Firmen aus dem Dump extrahiert werden.
Diese werden anschließend in einer Textdate names "title-categories-map.txt" abgelegt.

In [4]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*') # Regex-Kriterium für Erkennung von "Kategorie" Feld, welches sich unten auf jeder Wikipedia-Seite befindet.
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller'] # Sollten keywords dieser Liste in dem "Kategorien"-Feld enthalten sein, liefert dies Hinweise darauf, dass es sich bei dem Thema des Artikels um ein Unternehmen handelt.
uncertainty_markers = ['unternehmens', 'Unternehmens'] # Allerding gibt es auch Ausschlusskriterien, die zunächst nur auf mögliche Unsicherheiten hinweisen: könnte es sich in der Kategorie z.B. um einen Untenehmensberater handeln und nicht direkt ein Unternehmen?
words_re = re.compile("|".join(list_of_words)) # Erstellung des Regex-Musters mit Liste
uncertainty_re = re.compile("|".join(uncertainty_markers)) # Erstellung des Regex-Musters mit Liste

with open("../data/title-categories-map.txt", mode="w") as outfile: # Eine Textdatei wird im Schreibemodus erstellt oder geöffnet und bildet Ablage für erkannte Firmen.
    for i, page in enumerate(tqdm(dump)): # Iteration über Seiten im Dump
        try: # Try-Catch-Block, damit einzelne Fehler nicht zu Abbruch des Schreibprozesses führen - gerade bei großem Dump sehr wichtig
            line = "" # Erstellung der Lines, welche später in Textdatei abgebildet werden.
            line += page.title + "," # Der Titel der Wikipediaseite bildet das erste Attribut der Zeile und wird per Komma separiert
            for revision in page: # Iteration über revisions
                for match in pattern.finditer(revision.text): # Anwendung des ersten Regex-Patterns, welches auf Kategorienfeld hindeutet oder nicht.
                    line += match.group(1) + "," 
            line = line.strip(",")
            #lines.append(line)
            if words_re.search(str(line)): # Anwendung des zweiten Regex-Patterns, um grob zu erkennen, ob es sich bei der seite um die eines Unternehmens handeln könnte, oder nicht.
                if uncertainty_re.search(str(line)): # Das nächste Regex-Pattern weist auf Unsicherheiten hin, da oft "Unternehmens" + "andere Wortteil", darauf hindeuten, dass Artikel selbst nur etwas, mit der Firma im Zusammenhang setehendes, thematisiert.
                    if not reduceUncertainty(page.title): # Um hier mehr Gewissheit zu erlangen werden in den vereinzelten Fällen die eigenen Wikipediaartikel der Entitätenuntersucht. True wird zurückgegeben, wenn es sich um Firma handeln könnte.
                        print(page.title,reduceUncertainty(page.title)) # True bei reduceUncertainty() weist auf Firma hin, in welchem Fall der for-Loop fortgesetzt wird über continue.
                        continue
                    else:
                        outfile.write(line+"\n") 
                else:
                        outfile.write(line+"\n")
        except:
            print("An exception occurred") # Hier wird ein, falls notwendig erweiterbarer Fehlerhinweis ausgegeben.
            continue

38it [00:00, 143.14it/s]

10 -  Liste von Unternehmen mit Namensherkunftserklärungen


178it [00:06, 39.92it/s]

11 -  Aldi


192it [00:08, 15.47it/s]

...

145578it [23:06, 103.97it/s]

10 -  Heino Wiese


147099it [23:16, 38.13it/s] 

10 -  Sparkassen-Finanzgruppe


155401it [23:57, 108.07it/s]


## C) Extraktion der Infobox

Der Aufbau einer Infobox eines Unternehmens in dessen Wikipedia Artikel ist streng definiert. Die Infobox beginnt mit zwei geschweiften Klammern und dem Typ des Eintrags, in diesem Fall {{{Infobox Unternehmen. Sie enthält maximal 14 Einträge: Name, Logo, Unternehmensform, ISIN, Gründungsdatum, Auflösungsdatum, Sitz, Leitung, Mitarbeiterzahl, Umsatz, Stand, Branche, Website. Erforderlich sind hiervon Unternehmensform sowie Sitz. 

Die Infobox wird mit folgendem Regex Pattern definiert: ((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}})).
Es wird der Inhalt nach {{Infobox Unternehmen bis einschließlich dem letzten Eintrag Homepage.*?}} ausgelesen. 

Verbesserungsvorschlag: Einträge einzeln auslesen, sodass nur die Parameter ausgegeben werden, die einen Wert besitzen.

In [ ]:
#getting the infobox of a company

infobox = re.compile('((?<={{Infobox Unternehmen.).*?(?<=Homepage).*?(}}))') #regex pattern gets text between keyword 'Infobox Unternehmen' and 'Homepage' 

with open("../data/infobox.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ''
        for revision in page: 
            revision = re.sub(r'\n', '', revision.text) #removing linebreaks 
            for match in infobox.finditer(revision):
                line += match.group(1) + "\n" 
                line = " ".join(line.split()) #if there are multiple whitespaces, all except for one get deleted 
                outfile.write(line+"\n")
    

In [ ]:
num_lines = sum(1 for line in open('../data/infobox.txt'))
print(num_lines)
#Output:697

Nun soll auch für Unternehmen ohne Infobox deren Namen sowie dessen Webseite ausgegeben werden. 

Hierfür wird im ersten Schritt geprüft, ob der Wikipedia Artikel des Unternehmens das Wort Infobox enthält. Dies wird mit folgendem Regex Pattern geprüft: (?!.*?Infobox)^.*$ 
Enthält der Artikel das Wort nicht, so erkennt das Regex Pattern den gesamten Artikel. 
Wird der Artikel als Unternehmensartikel klassifiziert und enthält er das Wort Infobox nicht, wird der Artikelname (=Unternehmensname) ausgegeben. 

Um nun auch die Webseite des Unternehmens ausgeben zu lassen, wird folgendes Regex Pattern angewandt: 
.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*
Dies gibt alle in der Sektion Weblinks eingetragnenen Webseiten aus, worunter oftmals auch die offizielle Unternehmenswebseite fällt. 

Verbesserungsvorschlag: Es werden viele Artikel ausgelesen, die die Kategorie Unternehmsart enthalten, aber keine Unternehmen sind. Diese sind oftmals allgemeine Informationsseiten und enthalten daher auch keine Infobox, weshalb sie vermehrt ausgelesen werden. 
Darüber hinaus wird für die Unternehmenswebseite die ganze Sektion der Weblinks ausgelesen, die oftmals auch andere weiterführende Links enthält. Eine Verbesserungsmöglichkeit bestände darin, den Namen des Unternehmens in den Links zu suchen und nur die Links, die den Namen enthalten, auszugegeben. 

In [ ]:
#Companies without infobox
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

no_infobox = re.compile('(?!.*?Infobox)^.*$') #gets the whole article if it doesn´t contain 'Infobox'
pattern_website = re.compile('.*((?<=Weblinks...).*?(?=...Einzelnachweise)).*') 


with open("../data/company_name&website.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title
        website = ""
        for revision in page:
            revision = re.sub(r'\n', '', revision.text) #removing the line breaks 
            for match in pattern.finditer(revision):
                line += match.group(1)  
        
        if words_re.search(str(line)):
            for match in no_infobox.finditer(revision): #getting companies without an infobox 
                for match in pattern_website.finditer(revision): #getting the weblinks which contain the company url 
                    outfile.write(page.title+"\n") #printing the page title = company name 
                    website += match.group(1)
                    outfile.write(website+"\n"+"\n")

In [ ]:
companies_without_infobox = sum((1/3) for line in open('../data/company_name&website.txt'))
print(companies_without_infobox)
#Output: 136 

Um nun eine Kurzbeschreibung des Unternehmens zu erhalten, wird der erste Satz des Artikels erfasst. Dieser enthält bei Unternehmen den Unternehmensnamen eingefasst in ''' '''. Aus diesem Grund wird folgendes Regex Pattern angewandt, um den ersten Satz auszulesen: 
((?<=\''' ).*?(?<=\.)+) 

In [ ]:
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
pattern = re.compile('.*\[\[Kategorie:(.*)\]\].*')
list_of_words = ['hersteller', 'unternehmen', 'Unternehmen', 'Hersteller']
words_re = re.compile("|".join(list_of_words))

pattern_firstsentence = re.compile('((?<=\''' ).*?(?<=\.))')
#Erster Satz fängt meistens mit dem Firmennamen in ''' ''' an. Dann wird der erste Punkt gesucht.

with open("../data/firstsentence.txt", mode="w") as outfile:
    for i, page in enumerate(tqdm(dump)):
        line = ""
        line += page.title
        firstsentence = ""
        for revision in page:
            for match in pattern.finditer(revision.text):
                line += match.group(1)  
        
        if words_re.search(str(line)):
            for match in pattern_firstsentence.finditer(revision.text):
                firstsentence += match.group(1)
                firstsentence.replace(r'\*', '')
            outfile.write(page.title+"\n")
            outfile.write(firstsentence+"\n"+"\n")

In [ ]:
firstsentences = sum((1/3) for line in open('../data/firstsentence.txt'))
print(firstsentences)
#Output: 1570